# Cpx-Liquid Geothermometer for High-Fe, Low-Al Clinopyroxene from High-Silica Systems

_For details see: Brugman and Till (2019) American Mineralogist_

**Requires Python v3.6 or higher and Pandas**

___

**For use only with clinopyroxene with Mg# ≤ 65 and Al2O3 ≤ 7 wt% from a bulk rock with SiO2 ≥ 70 wt%**

Enter paired clinopyroxene and liquid oxide compositions via an Excel workbook (inputtemplate.xlsx).

• Ideal scenario: for liquid, please use data from the glass adjacent to your clinopyroxene rim analysis.

• Less idea scenario: if paired clinopyroxene rim and adjacent glass analyses are not available, an average glass composition may be used.

• Desperate scenario: if no glass measurements are available, use whole rock as a last-resort only—whole rock and glass oxides are not the same, particularly in the oxides necessary to use the geothermometer. For example, a temperature calculated for Bishop Tuff samples using whole rock is > 60°C higher (> 3x the SEE) than a temperature calculated correctly with glass.


In [1]:
# import dependencies
import pandas as pd
import datetime

## Input cpx and liquid compositions

In [3]:
# enter path or leave empty for same directory as this .ipynb file
path = "" 

# .xlsx file with columns: distance [m], measured, init
inputfilename = 'HighSiFeLowAlcpx_therm_inputtemplate' 
# use pandas to read in the excel file - kind of overkill, but ¯\_(ツ)_/¯
df = pd.read_excel(path + inputfilename + ".xlsx", skiprows = 2)

df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,CPX Original wt% — input any negative values as 0,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Locality,Sample,Reference,SiO2,TiO2,Al2O3,Cr2O3,FeO (tot),MnO,MgO,...,TiO2,Al2O3,Cr2O3,FeO (tot),MnO,MgO,CaO,Na2O,K2O,TOTAL
1,Bishop Tuff,AB-6202,Gardner et al. 2014,52.37,0.19,0.63,0,12.34,0.69,12.57,...,0.13,12.27,0,0.54,0.02,0.08,0.19,3.7,5.17,100
2,Huckleberry Ridge Tuff,YS-7 (rim),"Sisson, 1991",48.3,0.16,0.61,0,27.3,0.86,3.1,...,0.07,11.62,0,1.39,0.06,0.02,0.52,3.6,4.9,95.48
3,Huckleberry Ridge Tuff,YS-8 (rim),"Sisson, 1991",48.5,0.23,0.58,0,27.4,0.85,3.44,...,0.09,11.86,0,1.41,0.09,0.03,0.55,3.6,4.98,97.91
4,South Biscuit Basin,BS132,"Girard & Stix, 2010",51.44,0.19,0.68,0,16.42,0.83,10.68,...,0.14,11.63,0,0.36,0.02,0.02,0.44,3.27,5.44,95.04


## Thermometry code (do not edit)

In [7]:
# rename columns to be more meaningful
df.rename(index=str, columns={df.columns[3]: "cSiO2", df.columns[4]: "cTiO2", df.columns[5]: "cAl2O3", df.columns[6]: "cCr2O3", df.columns[7]: "cFeO", df.columns[8]: "cMnO", df.columns[9]: "cMgO", df.columns[10]: "cCaO", df.columns[11]: "cNa2O", df.columns[12]: "cK2O", df.columns[13]: "cTOTAL", df.columns[14]: "lSiO2", df.columns[15]: "lTiO2", df.columns[16]: "lAl2O3", df.columns[17]: "lCr2O3", df.columns[18]: "lFeO", df.columns[19]: "lMnO", df.columns[20]: "lMgO", df.columns[21]: "lCaO", df.columns[22]: "lNa2O", df.columns[23]: "lK2O", df.columns[24]: "lTOTAL"}, inplace=True)

# data validation
if ((df.cAl2O3 > 7.0).any()).any():
    print 'ALERT: This thermometer is not recommended for use with clinopyroxene with >7 wt% Al2O3'
    import sys
    sys.exit(0)

if ((100*(df.cMgO/40.311)/((df.cFeO/71.846)+(df.cMgO/40.311)) > 65.0).any()).any():
    print 'ALERT: This thermometer is not recommended for use with clinopyroxene with Mg# > 65'
    import sys
    sys.exit(0)

# convert df to list of dictionaries
oxides = df.to_dict('records')

# create dict of molecular masses
molemass = {'SiO2': 60.0843, 'TiO2': 79.8788, 'Al2O3': 101.961, 'Cr2O3': 151.9982, 'FeO': 71.8464, 'MnO': 70.9375, 'MgO': 40.3044, 'CaO': 56.0774, 'Na2O': 61.9789, 'K2O': 94.196}

# calculate components, cation fractions, etc. and then T
for i in oxides:
    # normalize oxides to 100
    i['cSiO2'] = 100*i['cSiO2']/i['cTOTAL']
    i['cTiO2'] = 100*i['cTiO2']/i['cTOTAL']
    i['cAl2O3'] = 100*i['cAl2O3']/i['cTOTAL']
    i['cCr2O3'] = 100*i['cCr2O3']/i['cTOTAL']
    i['cFeO'] = 100*i['cFeO']/i['cTOTAL']
    i['cMnO'] = 100*i['cMnO']/i['cTOTAL']
    i['cMgO'] = 100*i['cMgO']/i['cTOTAL']
    i['cCaO'] = 100*i['cCaO']/i['cTOTAL']
    i['cNa2O'] = 100*i['cNa2O']/i['cTOTAL']
    i['cK2O'] = 100*i['cK2O']/i['cTOTAL']
    i['cTOTAL'] = 100
    i['lSiO2'] = 100*i['lSiO2']/i['lTOTAL']
    i['lTiO2'] = 100*i['lTiO2']/i['lTOTAL']
    i['lAl2O3'] = 100*i['lAl2O3']/i['lTOTAL']
    i['lCr2O3'] = 100*i['lCr2O3']/i['lTOTAL']
    i['lFeO'] = 100*i['lFeO']/i['lTOTAL']
    i['lMnO'] = 100*i['lMnO']/i['lTOTAL']
    i['lMgO'] = 100*i['lMgO']/i['lTOTAL']
    i['lCaO'] = 100*i['lCaO']/i['lTOTAL']
    i['lNa2O'] = 100*i['lNa2O']/i['lTOTAL']
    i['lK2O'] = 100*i['lK2O']/i['lTOTAL']
    i['lTOTAL'] = 100
    
    # cpx: calculate mole proportions
    i['mcSiO2'] = i['cSiO2']/molemass['SiO2']
    i['mcTiO2'] = i['cTiO2']/molemass['TiO2']
    i['mcAl2O3'] = i['cAl2O3']/molemass['Al2O3']
    i['mcCr2O3'] = i['cCr2O3']/molemass['Cr2O3']
    i['mcFeO'] = i['cFeO']/molemass['FeO']
    i['mcMnO'] = i['cMnO']/molemass['MnO']
    i['mcMgO'] = i['cMgO']/molemass['MgO']
    i['mcCaO'] = i['cCaO']/molemass['CaO']
    i['mcNa2O'] = i['cNa2O']/molemass['Na2O']
    i['mcK2O'] = i['cK2O']/molemass['K2O']
    i['mcTOTAL'] = i['mcSiO2'] + i['mcTiO2'] + i['mcAl2O3'] + i['mcCr2O3'] + i['mcFeO'] + i['mcMnO'] + i['mcMgO'] + i['mcCaO'] + i['mcNa2O'] + i['mcK2O']
    
    # cpx: calculate number of oxygens
    i['ocSiO2'] = i['mcSiO2']*2
    i['ocTiO2'] = i['mcTiO2']*2
    i['ocAl2O3'] = i['mcAl2O3']*3
    i['ocFeO'] = i['mcFeO']
    i['ocMnO'] = i['mcMnO']
    i['ocMgO'] = i['mcMgO']
    i['ocCaO'] = i['mcCaO']
    i['ocNa2O'] = i['mcNa2O']
    i['ocK2O'] = i['mcK2O']
    i['ocCr2O3'] = i['mcCr2O3']
    i['ocTOTAL'] = i['ocSiO2'] + i['ocTiO2'] + i['ocAl2O3'] + i['ocCr2O3'] + i['ocFeO'] + i['ocMnO'] + i['ocMgO'] + i['ocCaO'] + i['ocNa2O'] + i['ocK2O']
    i['oxnormfactor'] = 6/i['ocTOTAL']
    
    # cpx: calculate cations on the basis of 6 oxygens
    i['cSi'] = i['mcSiO2']*i['oxnormfactor']
    i['cTi'] = i['mcTiO2']*i['oxnormfactor']
    i['cAltot'] = i['mcAl2O3']*i['oxnormfactor']*2
    i['cAl4'] = 2 - i['cSi']
    if i['cAltot'] - i['cAl4'] < 0:
        i['cAl6'] = 0
    else:
        i['cAl6'] = i['cAltot'] - i['cAl4']
    i['cCr'] = i['mcCr2O3']*i['oxnormfactor']*2
    i['cFe'] = i['mcFeO']*i['oxnormfactor']
    i['cMn'] = i['mcMnO']*i['oxnormfactor']
    i['cMg'] = i['mcMgO']*i['oxnormfactor']
    i['cCa'] = i['mcCaO']*i['oxnormfactor']
    i['cNa'] = i['mcNa2O']*i['oxnormfactor']*2
    i['cK'] = i['mcK2O']*i['oxnormfactor']*2

    # cpx: calculate cpx components
    if i['cNa'] < i['cAl6']:
        i['Jd'] = i['cNa']
        i['CaTs'] = i['cAl6'] - i['cNa']
    else:
        i['Jd'] = i['cAl6']
        i['CaTs'] = 0
    if i['cAl4'] > i['CaTs']:
        i['CaTi'] = (i['cAl4'] - i['CaTs'])/2
    else:
        i['CaTi'] = 0
    i['CrCaTs'] = i['cCr']/2
    if i['cCa'] - i['CaTi'] - i['CaTs'] - i['CrCaTs'] > 0:
        i['DiHd1996'] = i['cCa'] - i['CaTi'] - i['CaTs'] - i['CrCaTs']
    else:
        i['DiHd1996'] = 0
    i['DiHd2003'] = i['cCa'] - i['CaTi'] - i['CaTs'] - i['CrCaTs']
    i['EnFs'] = ((i['cFe'] + i['cMg']) - i['DiHd1996'])/2
    i['compsum1996'] = i['Jd'] + i['CaTs'] + i['CaTi'] + i['CrCaTs'] + i['DiHd1996'] + i['EnFs']
    i['compsum2003'] = i['Jd'] + i['CaTs'] + i['CaTi'] + i['CrCaTs'] + i['DiHd2003'] + i['EnFs']
    i['acpxEn'] = (1 - i['cCa'] - i['cNa'] - i['cK'])*(1 - 0.5*(i['cAltot'] + i['cNa'] + i['cK'] + i['cCr']))

    # liq: calculate cation proportions
    i['clSiO2'] = i['lSiO2']/molemass['SiO2']
    i['clTiO2'] = i['lTiO2']/molemass['TiO2']
    i['clAl2O3'] = i['lAl2O3']*2/molemass['Al2O3']
    i['clCr2O3'] = i['lCr2O3']/molemass['Cr2O3']
    i['clFeO'] = i['lFeO']/molemass['FeO']
    i['clMnO'] = i['lMnO']/molemass['MnO']
    i['clMgO'] = i['lMgO']/molemass['MgO']
    i['clCaO'] = i['lCaO']/molemass['CaO']
    i['clNa2O'] = i['lNa2O']*2/molemass['Na2O']
    i['clK2O'] = i['lK2O']*2/molemass['K2O']
    i['clTOTAL'] = i['clSiO2'] + i['clTiO2'] + i['clAl2O3'] + i['clCr2O3'] + i['clFeO'] + i['clMnO'] + i['clMgO'] + i['clCaO'] + i['clNa2O'] + i['clK2O']

    # liq: calculate cation fractions
    i['flSiO2'] = i['clSiO2']/i['clTOTAL']
    i['flTiO2'] = i['clTiO2']/i['clTOTAL']
    i['flAl2O3'] = i['clAl2O3']/i['clTOTAL']
    i['flCr2O3'] = i['clCr2O3']/i['clTOTAL']
    i['flFeO'] = i['clFeO']/i['clTOTAL']
    i['flMnO'] = i['clMnO']/i['clTOTAL']
    i['flMgO'] = i['clMgO']/i['clTOTAL']
    i['flCaO'] = i['clCaO']/i['clTOTAL']
    i['flNa2O'] = i['clNa2O']/i['clTOTAL']
    i['flK2O'] = i['clK2O']/i['clTOTAL']
    i['flTOTAL'] = i['flSiO2'] + i['flTiO2'] + i['flAl2O3'] + i['flCr2O3'] + i['flFeO'] + i['flMnO'] + i['flMgO'] + i['flCaO'] + i['flNa2O'] + i['flK2O']
    
    i['T'] = 300*(-1.8946098 - 0.6010197*i['CaTs'] - 0.1856423*i['DiHd2003'] + 4.71248858*i['flSiO2'] + 77.5861878*i['flTiO2'] + 10.8503727*i['flFeO'] + 33.6303471*i['flMgO'] + 15.4532888*i['flCaO'] + 15.6390115*i['flK2O'])
    
    print("%.2f C  %s %s (%s)" % (i['T'], i['Locality'], i['Sample'], i['Reference']))


# ----- output output new spreadsheet with T in new column -----

# make filename 
today = datetime.date.today()
todaysdate = today.strftime('%Y%m%d')

savefile = inputfilename + '_out_' + todaysdate + '.xlsx';

# output dictionary to Excel, but only certain items
new_df = pd.DataFrame(oxides)
new_df = new_df[['Locality','Sample', 'Reference', 'T']]
new_df.columns.values[3] = 'T [C]'
    
writer = pd.ExcelWriter(path + savefile)
new_df.to_excel(writer, 'Sheet1', index = False)

print '\nResults have been saved to ' + path + savefile + '\n'



748.90 C  Bishop Tuff AB-6202  (Gardner et al. 2014)
772.33 C  Bandelier Tuff Tshirege bottom (Warshaw & Smith, 1988)

Results have been saved to /Users/kara/Box Sync/1. Scaup Lake/6. Thermometry/Putirka px-liq v2/New High Si Fe Low Al thermometer/4. new regression trials with other components/1. Main final components/trial A for publication/inputtest_20180402_out_20181003.xlsx

